In [ ]:
import json

languages = ['go', 'java', 'javascript', 'php', 'python', 'ruby']

for language in languages:
    jsonl_file_path = f'{language}/train.jsonl'

    repo_counts = {}

    with open(jsonl_file_path, 'r') as file:
        for line in file:
            json_object = json.loads(line)
            
            if "repo" in json_object:
                repo_value = json_object["repo"]
                
                if repo_value in repo_counts:
                    repo_counts[repo_value] += 1
                else:
                    repo_counts[repo_value] = 1

    sorted_repo_counts = dict(sorted(repo_counts.items(), key=lambda x: x[0]))

    for repo, count in sorted_repo_counts.items():
        print(f"repo: {repo}, count: {count}")

In [ ]:
import json
import os

languages = ['go', 'java', 'javascript', 'php', 'python', 'ruby']

for language in languages:
    jsonl_file_path = f'{language}/train.jsonl'
    output_jsonl_file = f'{language}/train_sorted.jsonl'

    with open(jsonl_file_path, 'r') as input_file:
        lines = input_file.readlines()

    sorted_lines = sorted(lines, key=lambda line: json.loads(line).get("repo", ""))

    with open(output_jsonl_file, 'w') as output_file:
        for line in sorted_lines:
            output_file.write(line)


In [ ]:
import json
import os

languages = ['go', 'java', 'javascript', 'php', 'python', 'ruby']

for language in languages:
    jsonl_file_path = f'{language}/train_sorted.jsonl'
    output_folder = f'{language}/split_jsonl_files'

    num = 0

    os.makedirs(output_folder, exist_ok=True)

    user_objects = {}

    with open(jsonl_file_path, 'r') as input_file:
        for line in input_file:
            num = num + 1
            json_object = json.loads(line)

            repo_value = json_object.get("repo", "")

            username = repo_value.split('/')[0]

            if username not in user_objects:
                user_objects[username] = []
            user_objects[username].append(json_object)

    jsons_per_subset = num // 10

    current_subset = []
    file_number = 0

    print(f"Language: {language}")
    print(f"JSONs per Subset: {jsons_per_subset}")

    for username, user_jsons in user_objects.items():
        current_subset.extend(user_jsons)
        if len(current_subset) >= jsons_per_subset:
            output_file_path = os.path.join(output_folder, f'subset_{file_number}.jsonl')
            with open(output_file_path, 'w') as output_file:
                for json_object in current_subset:
                    json.dump(json_object, output_file)
                    output_file.write('\n')

            print(f"Subset {file_number}: {len(current_subset)} JSONs")

            current_subset = []
            file_number += 1

    if current_subset:
        output_file_path = os.path.join(output_folder, f'subset_{file_number}.jsonl')
        with open(output_file_path, 'w') as output_file:
            for json_object in current_subset:
                json.dump(json_object, output_file)
                output_file.write('\n')
        
        print(f"Subset {file_number}: {len(current_subset)} JSONs")
